# PATMOS-x v06r00 time series from monthly means

Monthly means for PATMOS-x v06r00 data have been computed with the script [PATMOS-x_monthlymean.py](../scripts/PATMOS-x_monthlymean.py). The monthly means are computed at the pixel level. That is, the monthly mean for a given variable is a 3D array of coordinates `(time, latitude, longitude)`. Here, `time` is a `datetime64[ns]` which indicates the month.

The monthly means have been computed for the following variables:
- `cloud_probability`
- `cloud_fraction`
- `tiwp`
- `tiwp_mixed`

Notes:
- Concerning `cloud_fraction`, based on https://doi.org/10.3390/rs8060511, which is not the [official documentation](https://www.ncei.noaa.gov/pub/data/sds/cdr/CDRs/AVHRR-HIRS_Cloud_Properties_PATMOS-x/AlgorithmDescription/AlgorithmDescription_ACHA_01B-1d.pdf) (it says very little), it is obtained by:
  1. Centering an array of 3x3 pixels of the native satellite on the PATMOS-x pixel.
  2. Determining if the native pixel is cloudy based on `cloud_probability > 0.5` computed for the native pixel.
  3. Taking the average of the 3x3 boolean array.

  The `cloud_fraction` can be observed as a mean cloud probability ([source](https://doi.org/10.1175/JAMC-D-11-02.1)).

- `tiwp` is obtained by considering the cloud water path of those pixels which are indicated to have an ice cloud phase.

- `tiwp_mixed` is obtained by considering the cloud water path of those pixels which are indicated to have a mixed cloud phase or ice cloud phase.

- The cloud water paths are computed using the DCOMP (daytime-only) algorithm.

- PATMOS-x has a constant spatial grid.

- The monthly means files contain the variables `var_count`, e.g. `cloud_fraction_count`, which indicate how many values were used to compute the average for every pixel.

Based on the last point, there are two options when summarizing the values as one value per month:
- Type I average: average of averages.
  - 1: Compute the monthly average for each pixel
  - 2: Weighted average of these pixelwise monthly averages (weights: $\cos(\text{latitude})$)
- Type II average: average all values.
  - 1: Weighted average all values in the month (weights: $\cos(\text{latitude})$)

I am not aware if 'type I' and 'type II' are standards. The nomenclature is irrelevant.

In [ ]:
# Load libraries
import datetime
from pathlib import Path

import cartopy.crs as ccrs
import cmocean
from dask.diagnostics import ProgressBar
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np
import tqdm
import xarray as xr

plt.style.use("../ccic.mplstyle")

In [ ]:
# Get monthly means file
mm_files = sorted(list(Path('/scratch/ccic_record/data/PATMOS-x/means/monthly_means').glob('PATMOS-x_v06-monthlymean_*nc')))

# Assert all files expected are there
month_start = mm_files[0].stem.split('_')[-1]
month_end = mm_files[-1].stem.split('_')[-1]
datetime_start = datetime.datetime.strptime(month_start, '%Y%m')
datetime_end = datetime.datetime.strptime(month_end, '%Y%m')
n_months_expected = (datetime_end.year - datetime_start.year) * 12 + datetime_end.month - datetime_start.month + 1
n_months_found = len(mm_files)
assert n_months_expected == n_months_found

In [ ]:
# Open the files (not loaded into memory)
ds = xr.open_mfdataset(mm_files)

In [ ]:
# Prepare the mask to use by resampling to the PATMOS-x coordinates
mask = xr.load_dataset('/scratch/ccic_record/data/mask_24.nc').mask

fig, axs = plt.subplots(ncols=2, subplot_kw=dict(projection=ccrs.PlateCarree()))

lons, lats = np.meshgrid(mask.longitude.data, mask.latitude.data)
m0 = axs[0].pcolormesh(lons, lats, mask, transform=ccrs.PlateCarree())
axs[0].coastlines()
axs[0].set_global()
axs[0].set_title('Mask before interpolation')
fig.colorbar(m0, ax=axs[0], shrink=0.2)


mask = mask.astype(int).interp(
    {
        'latitude': ds.latitude.data,
        'longitude': ds.longitude.data
    },
    method='nearest'
) == 1 # .astype(bool) doesn't work, since NaNs are converted to True


lons, lats = np.meshgrid(mask.longitude.data, mask.latitude.data)
m1 = axs[1].pcolormesh(lons, lats, mask, transform=ccrs.PlateCarree(), rasterized=True)
axs[1].coastlines()
axs[0].set_global()
axs[1].set_title('Mask after interpolation')
fig.colorbar(m1, ax=axs[1], shrink=0.2)

fig.savefig('../figures/PATMOSx_mask.pdf', bbox_inches='tight')
fig.savefig('../figures/PATMOSx_mask.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# Save the mask to a results dataset
ds_results = xr.Dataset(
    data_vars={
        'mask': (('latitude', 'longitude'), mask.data)
    },
    coords={
        'latitude': ('latitude', mask.latitude.data),
        'longitude': ('longitude', mask.longitude.data)
    }
)

## Average maps of the different variables

In [ ]:
# First: add values to results dataset

# Take into account the number of points available for each pixel
full_weighted_average = lambda var: ds[var].weighted(ds[f"{var}_count"]).mean(dim='time', skipna=True).data

ds_results = ds_results.merge(
    xr.Dataset(
        data_vars={
            'cloud_probability_fullaverage': (
                ('latitude', 'longitude'),
                full_weighted_average('cloud_probability'),
                {'period': f'{month_start}-{month_end}'}
            ),
            'cloud_fraction_fullaverage': (
                ('latitude', 'longitude'),
                full_weighted_average('cloud_fraction'),
                {'period': f'{month_start}-{month_end}'}
            ),
            'tiwp_fullaverage': (
                ('latitude', 'longitude'),
                full_weighted_average('tiwp'),
                {
                    'period': f'{month_start}-{month_end}',
                    'units': ds.tiwp.units
                }
            ),
            'tiwp_mixed_fullaverage': (
                ('latitude', 'longitude'),
                full_weighted_average('tiwp_mixed'),
                {
                    'period': f'{month_start}-{month_end}',
                    'units': ds.tiwp_mixed.units
                }
            ),
            
        },
        coords={
            'latitude': ('latitude', ds.latitude.data),
            'longitude': ('longitude', ds.longitude.data)
        }
    )
)

In [ ]:
# Second: compute and plot
fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw=dict(projection=ccrs.PlateCarree()))

step = 10

var = 'cloud_probability'
m_cloud_probability = axs[0,0].contourf(
    ds_results.longitude.data[::step],
    ds_results.latitude.data[::step],
    ds_results[f'{var}_fullaverage'].data[::step, ::step],
    transform=ccrs.PlateCarree(),
    vmin=0,
    vmax=1,
    levels=11
)
m_cloud_probability.set_rasterized(True)
fig.colorbar(m_cloud_probability, ax=axs[0,0], shrink=0.5)
axs[0,0].set_title(rf'\verb+{var}+', loc='center')

var = 'cloud_fraction'
m_cloud_fraction = axs[0,1].contourf(
    ds_results.longitude.data[::step],
    ds_results.latitude.data[::step],
    ds_results[f'{var}_fullaverage'].data[::step, ::step],
    transform=ccrs.PlateCarree(),
    vmin=0,
    vmax=1,
    levels=11
)
m_cloud_fraction.set_rasterized(True)
fig.colorbar(m_cloud_fraction, ax=axs[0,1], shrink=0.5)
axs[0,1].set_title(rf'\verb+{var}+', loc='center')

norm = LogNorm(1e-2, 1e2, clip=True)
levels = np.logspace(-2, 2, 11)

var = 'tiwp'
m_tiwp = axs[1,0].contourf(
    ds_results.longitude.data[::step],
    ds_results.latitude.data[::step],
    ds_results[f'{var}_fullaverage'].data[::step, ::step] * 1e-3, # Convert to kg/m2
    transform=ccrs.PlateCarree(),
    norm=norm,
    levels=levels
)
m_tiwp.set_rasterized(True)
fig.colorbar(m_tiwp, ax=axs[1,0], shrink=0.5, extend='both')
axs[1,0].set_title(rf'\verb+{var}+ [\si{{\kilo\gram\per\square\metre}}]', loc='center')

var = 'tiwp_mixed'
m_tiwp_mixed = axs[1,1].contourf(
    ds_results.longitude.data[::step],
    ds_results.latitude.data[::step],
    ds_results[f'{var}_fullaverage'].data[::step, ::step] * 1e-3, # Convert to kg/m2
    transform=ccrs.PlateCarree(),
    norm=norm,
    levels=levels
)
m_tiwp_mixed.set_rasterized(True)
fig.colorbar(m_tiwp_mixed, ax=axs[1,1], shrink=0.5, extend='both')
axs[1,1].set_title(rf'\verb+{var}+ [\si{{\kilo\gram\per\square\metre}}]', loc='center')

for i, ax in enumerate(axs.ravel()):
    ax.coastlines()
    ax.set_global()
    ax.set_title(f'({chr(97 + i)})', loc='left')

fig.suptitle(f'Average maps for period {month_start}-{month_end}')

fig.tight_layout()

fig.savefig(f'../figures/PATMOSx_expectedmaps_{month_start}-{month_end}.pdf', bbox_inches='tight')
fig.savefig(f'../figures/PATMOSx_expectedmaps_{month_start}-{month_end}.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw=dict(projection=ccrs.PlateCarree()))

step = 10

for i, (ax, var) in enumerate(zip(axs.ravel(), ['cloud_probability', 'cloud_fraction', 'tiwp', 'tiwp_mixed'])):
    ds_results = ds_results.merge(
        xr.Dataset(
            data_vars={
                f'{var}_fullcount': (
                    ('latitude', 'longitude'),
                    ds[f"{var}_count"].sum(dim="time", skipna=True).data,
                    {'period': f'{month_start}-{month_end}'}
                )
            },
            coords={
                'latitude': ('latitude', ds.latitude.data),
                'longitude': ('longitude', ds.longitude.data)
            }
        )
    )

    c = ax.contourf(
        ds_results.longitude.data[::step],
        ds_results.latitude.data[::step],
        ds_results[f"{var}_fullcount"].data[::step, ::step],
        transform=ccrs.PlateCarree()
    )
    c.set_rasterized(True)
    fig.colorbar(c, ax=ax, shrink=0.5)
    ax.coastlines()
    ax.set_global()
    ax.set_title(f'({chr(97 + i)})', loc='left')
    ax.set_title(rf'\verb+{var}+', loc='center')

fig.suptitle(f'Number of data points used in each pixel for period {month_start}-{month_end}')

fig.tight_layout()

fig.savefig(f'../figures/PATMOSx_countmaps_{month_start}-{month_end}.pdf', bbox_inches='tight')
fig.savefig(f'../figures/PATMOSx_countmaps_{month_start}-{month_end}.png', dpi=300, bbox_inches='tight')

plt.show()

What is the numerical difference between `cloud_probability` and `cloud_fraction`?

In [ ]:
diff = ds_results['cloud_probability_fullaverage'] - ds_results['cloud_fraction_fullaverage']

fig = plt.figure()
gs = fig.add_gridspec(nrows=2)

ax1 = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())
c = ax1.contourf(ds_results.longitude.data[::step], ds_results.latitude.data[::step], diff.data[::10,::10], vmin=-1, vmax=1, cmap='coolwarm', transform=ccrs.PlateCarree())
c.set_rasterized(True)
ax1.set_title(r'\verb+cloud_probability+ $-$ \verb+cloud_fraction+', loc='center')
fig.colorbar(c, ax=ax1, shrink=0.75)
ax1.set_global()
ax1.coastlines()

ax2 = fig.add_subplot(gs[1])
ax2.hist(diff.data.flatten(), bins=np.linspace(-1, 1, 500, endpoint=True), density=True)
ax2.set_xlabel(r'\verb+cloud_probability+ $-$ \verb+cloud_fraction+')
ax2.set_ylabel('probability density function')

fig.tight_layout()

fig.savefig(f'../figures/PATMOSx_diffCPCF_{month_start}-{month_end}.pdf', bbox_inches='tight')
fig.savefig(f'../figures/PATMOSx_diffCPCF_{month_start}-{month_end}.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# As before:
# 1st: (define) computations
# 2nd: plot

weights_latitude = np.cos(np.deg2rad(ds_results.latitude))
for var in ['cloud_probability', 'cloud_fraction', 'tiwp', 'tiwp_mixed']:
    single_weights = weights_latitude
    arr_var_single_weighted = ds[var].weighted(single_weights)
    arr_var_masked_single_weighted = ds[var].where(mask == True).weighted(single_weights)

    double_weights = weights_latitude * ds[f'{var}_count']
    arr_var_double_weighted = ds[var].weighted(double_weights)
    arr_var_masked_double_weighted = ds[var].where(mask == True).weighted(double_weights)

    ds_results = ds_results.merge(
        xr.Dataset(
            data_vars={
                f'{var}_lataverage': (
                    'time', arr_var_single_weighted.mean(dim=('longitude', 'latitude'), skipna=True).data
                ),
                f'{var}_lataverage_masked': (
                    'time', arr_var_masked_single_weighted.mean(dim=('longitude', 'latitude'), skipna=True).data
                ),
                f'{var}_latcountaverage': (
                    'time', arr_var_double_weighted.mean(dim=('longitude', 'latitude'), skipna=True).data
                ),
                f'{var}_latcountaverage_masked': (
                    'time', arr_var_masked_double_weighted.mean(dim=('longitude', 'latitude'), skipna=True).data
                ),
            },
            coords={
                'time': ds.time.data
            }
        )
    )

In [ ]:
# Save results to disk
ds_results.to_netcdf(
    f'/scratch/ccic_record/data/PATMOS-x/means/PATMOS-x_timeseries{month_start}-{month_end}_notebook_results.nc',
    encoding={
        var: {'zlib': True, 'complevel': 9}
        for var in ds_results
    }
)

In [ ]:
fig, axs = plt.subplots(nrows=4, figsize=(16,6))

ax_cloud, ax_cloud_legend, ax_tiwp, ax_tiwp_legend = axs

alpha = 0.3

for i, var in enumerate(['cloud_probability', 'cloud_fraction']):
    color = f'C{i*4}'
    ax_cloud.plot(
        ds_results.time.data,
        ds_results[f'{var}_lataverage'].data,
        color=color,
        label=f'{var} (type I)'
    )

    color = f'C{i*4+1}'
    ax_cloud.plot(
        ds_results.time.data,
        ds_results[f'{var}_lataverage_masked'].data,
        color=color,
        label=f'{var}, masked (type I)'
    )

    color = f'C{i*4+2}'
    ax_cloud.plot(
        ds_results.time.data,
        ds_results[f'{var}_latcountaverage'].data,
        color=color,
        label=f'{var} (type II)'
    )

    color = f'C{i*4+3}'
    ax_cloud.plot(
        ds_results.time.data,
        ds_results[f'{var}_latcountaverage_masked'].data,
        color=color,
        label=f'{var}, masked (type II)'
    )

ax_cloud.minorticks_on()
ax_cloud.grid(
    which='both',
    alpha=alpha,
    color='grey',
    ls='dotted'
)

ax_cloud_legend.set_axis_off()
ax_cloud_legend.legend(
    handles=ax_cloud.get_legend_handles_labels()[0],
    labels=ax_cloud.get_legend_handles_labels()[1],
    ncols=4,
    loc='center'
)
# ax_cloud.set_ylim(0, 1)

for i, var in enumerate(['tiwp', 'tiwp_mixed']):
    weights_count = ds[f'{var}_count'].astype(float)
    weights_count.data = 1 / np.where(weights_count == 0, np.nan, weights_count)
    weights = weights_latitude * weights_count
    weighted_var = ds[var].weighted(weights.fillna(0))
    weighted_var_masked = ds[var].where(mask == True).weighted(weights.fillna(0))

    color = f'C{i*4}'
    ax_tiwp.plot(
        ds_results.time.data,
        ds_results[f'{var}_lataverage'].data * 1e-3,
        color=color,
        label=f'{var} (type I)'
    )

    color = f'C{i*4+1}'
    ax_tiwp.plot(
        ds_results.time.data,
        ds_results[f'{var}_lataverage_masked'].data * 1e-3,
        color=color,
        label=f'{var}, masked (type I)'
    )

    color = f'C{i*4+2}'
    ax_tiwp.plot(
        ds_results.time.data,
        ds_results[f'{var}_latcountaverage'].data * 1e-3,
        color=color,
        label=f'{var} (type II)'
    )

    color = f'C{i*4+3}'
    ax_tiwp.plot(
        ds_results.time.data,
        ds_results[f'{var}_latcountaverage_masked'].data * 1e-3,
        color=color,
        label=f'{var}, masked (type II)'
    )

ax_tiwp.set_ylabel(r'\si{\kilo\gram\per\square\metre}')

ax_tiwp.minorticks_on()
ax_tiwp.grid(
    which='both',
    alpha=alpha,
    color='grey',
    ls='dotted'
)

ax_tiwp_legend.set_axis_off()
ax_tiwp_legend.legend(
    handles=ax_tiwp.get_legend_handles_labels()[0],
    labels=ax_tiwp.get_legend_handles_labels()[1],
    ncols=4,
    loc='center'
)
# ax_tiwp.set_ylim(5e-2, 1e0)
ax_tiwp.set_yscale('log')

fig.suptitle('See notes for the difference between type I and type II')

fig.tight_layout()

fig.savefig(f'../figures/PATMOSx_timeseries_{month_start}-{month_end}.pdf', bbox_inches='tight')
fig.savefig(f'../figures/PATMOSx_timeseries_{month_start}-{month_end}.png', dpi=300, bbox_inches='tight')

plt.show()

Reminder on type I and type II:
- Type I average: average of averages.
  - 1: Compute the monthly average for each pixel
  - 2: Weighted average of these pixelwise monthly averages (weights: $\cos(\text{latitude})$)
- Type II average: average all values.
  - 1: Weighted average all values in the month (weights: $\cos(\text{latitude})$)